In [240]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import itertools
import glob
import pickle
import matplotlib.pyplot as plt
import qmin
import os

In [241]:
def load_classification_model():
    modelsaved = './dashboard/model_py/allmodels.pkl'
    with open(modelsaved, 'rb') as f:
        model = pickle.load(f)
    return model

model = load_classification_model()

datapath = '../data_train/Anfibolios_Li-et-al-2020.xlsx'
df = pd.read_excel(datapath)
df['FEOT'] = df['FeOtot']

dic = {}
for i in range(len(df.columns)):
    dic[df.columns[i]] = str(df.columns[i]).strip().upper()
df = df.rename(columns=dic)
df_anf = df
print(df.columns)
df

/Users/marcosferreira/anaconda3/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning:

Trying to unpickle estimator DecisionTreeClassifier from version 0.22.1 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.

/Users/marcosferreira/anaconda3/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning:

Trying to unpickle estimator RandomForestClassifier from version 0.22.1 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.



Index(['SUBSET', 'REFERENCE', 'LABEL', 'HOST ROCK/ORIGIN', 'METHODS', 'SIO2',
       'TIO2', 'AL2O3', 'CR2O3', 'FE2O3', 'FEO', 'FEOTOT', 'MNO', 'MGO', 'CAO',
       'NA2O', 'K2O', 'F', 'CL', 'LI2O', 'NIO', 'ZNO', 'T.SI.REF', 'T.AL.REF',
       'T.TI.REF', 'T.SUM.REF', 'C.AL.REF', 'C.TI.REF', 'C.FE3.REF',
       'C.CR.REF', 'C.NI.REF', 'C.ZN.REF', 'C.MN2.REF', 'C.MG.REF',
       'C.FE2.REF', 'C.LI.REF', 'SUM C.REF', 'B.MN2.REF', 'B.FE2.REF',
       'B.MG.REF', 'B.LI.REF', 'B.CA.REF', 'B.NA .REF', 'B.SUM.REF',
       'A.LI.REF', 'A.CA.REF', 'A.NA.REF', 'A.K.REF', 'A.SUM.REF', 'W.OH.REF',
       'W.F.REF', 'W.CL.REF', 'W.I.REF', 'W.O.REF', 'W.SUM.REF', 'FEOT'],
      dtype='object')


,SUBSET,REFERENCE,LABEL,HOST ROCK/ORIGIN,METHODS,SIO2,TIO2,AL2O3,CR2O3,FE2O3,...,A.NA.REF,A.K.REF,A.SUM.REF,W.OH.REF,W.F.REF,W.CL.REF,W.I.REF,W.O.REF,W.SUM.REF,FEOT
0,Training set,"Cámara and Oberti, 2005",4,various,"EPMA, SREF,SIMS",60.41,0.01,11.26,0.01,1.97,...,0.020,0.000,0.020,1.930,0.070,NaN,NaN,NaN,2.0,10.802626
1,Training set,"Cámara and Oberti, 2005",5,various,"EPMA, SREF,SIMS",60.42,0.01,8.54,0.03,6.05,...,0.030,0.000,0.030,1.910,0.090,NaN,NaN,NaN,2.0,11.363850
2,Training set,"Cámara and Oberti, 2005",6,various,"EPMA, SREF,SIMS",58.84,0.02,8.21,0.01,6.72,...,0.020,0.000,0.020,1.870,0.130,NaN,NaN,NaN,2.0,11.606723
3,Test set,"Cámara and Oberti, 2005",7,various,"EPMA, SREF,SIMS",59.93,0.02,8.42,0.02,5.83,...,0.040,0.000,0.040,1.900,0.100,NaN,NaN,NaN,2.0,11.305892
4,Training set,"Hawthorne et al., 1996a",H(1),granitic pegmatite,"EPMA, SREF, SIMS",50.31,0.57,0.63,NaN,12.00,...,0.680,0.316,0.996,0.855,1.145,NaN,NaN,NaN,2.0,32.937720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,Training set,"Oberti et al., 2009a",64715,Li-pegmatite,"EPMA, SREF, SIMS",59.81,0.09,12.66,NaN,NaN,...,0.680,NaN,0.680,0.970,1.030,NaN,NaN,NaN,2.0,10.320000
62,Training set,"Oberti et al., 2009b",1122,agpaitic nepheline-syenite,"EPMA, SREF",58.16,0.32,7.06,NaN,6.05,...,0.650,0.430,1.080,0.530,1.470,NaN,NaN,0.0,2.0,8.713850
63,Training set,"Oberti et al., 2015",AMNH H32734,Jade Mine,"EPMA, SREF",51.74,0.14,7.38,2.48,2.82,...,0.848,0.043,0.891,1.674,0.026,NaN,NaN,0.3,2.0,4.090000
64,Training set,"Oberti et al., 2018a",762,silicocarbonatitic,"EPMA, SREF",50.46,0.74,4.02,NaN,2.84,...,0.550,0.320,0.870,0.770,1.230,NaN,NaN,0.0,2.0,12.240000


In [242]:
for i in df.columns:
#     if i == 'FEO':
#         df = df.rename(columns={'FEO': 'FEOT'})
#         continue
    if i not in model['Train Features']:
        df = df.drop(columns=i)
print(df.columns,model['Train Features'], len(df.columns),len(model['Train Features']), df_anf.columns)
df = df.fillna(0)


Index(['SIO2', 'TIO2', 'AL2O3', 'CR2O3', 'MNO', 'MGO', 'CAO', 'NA2O', 'K2O',
       'F', 'CL', 'NIO', 'ZNO', 'FEOT'],
      dtype='object') ['SIO2', 'TIO2', 'AL2O3', 'CR2O3', 'FEOT', 'CAO', 'MGO', 'MNO', 'K2O', 'NA2O', 'P2O5', 'F', 'CL', 'NIO', 'CUO', 'COO', 'ZNO', 'PBO', 'S', 'ZRO2', 'AS'] 14 21 Index(['SUBSET', 'REFERENCE', 'LABEL', 'HOST ROCK/ORIGIN', 'METHODS', 'SIO2',
       'TIO2', 'AL2O3', 'CR2O3', 'FE2O3', 'FEO', 'FEOTOT', 'MNO', 'MGO', 'CAO',
       'NA2O', 'K2O', 'F', 'CL', 'LI2O', 'NIO', 'ZNO', 'T.SI.REF', 'T.AL.REF',
       'T.TI.REF', 'T.SUM.REF', 'C.AL.REF', 'C.TI.REF', 'C.FE3.REF',
       'C.CR.REF', 'C.NI.REF', 'C.ZN.REF', 'C.MN2.REF', 'C.MG.REF',
       'C.FE2.REF', 'C.LI.REF', 'SUM C.REF', 'B.MN2.REF', 'B.FE2.REF',
       'B.MG.REF', 'B.LI.REF', 'B.CA.REF', 'B.NA .REF', 'B.SUM.REF',
       'A.LI.REF', 'A.CA.REF', 'A.NA.REF', 'A.K.REF', 'A.SUM.REF', 'W.OH.REF',
       'W.F.REF', 'W.CL.REF', 'W.I.REF', 'W.O.REF', 'W.SUM.REF', 'FEOT'],
      dtype='object')


In [243]:
import plotly.express as px
import plotly.graph_objects as go

def create_Model_Regression(labels):
    features = np.array(df)
    train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.2)
    
    df['split'] = 'train'
    df.loc[test_features.index, 'split'] = 'test'
    # print('Training Features Shape:', train_features.shape)
    # print('Training Labels Shape:', train_labels.shape)
    # print('Testing Features Shape:', test_features.shape)
    # print('Testing Labels Shape:', test_labels.shape)
    rf = RandomForestRegressor(n_estimators = 100)
    # Train the model on training data
    rf.fit(train_features, train_labels);
    # Use the forest's predict method on the test data
    predictions = rf.predict(test_features)
    # Calculate the absolute errors
    errors = abs(predictions - test_labels)
    # Print out the mean absolute error (mae)
    print('Mean Absolute Error:', round(np.mean(errors), 2), 'Mols.')
    
    df[elements] = df_ref[elements]
    df[elements + '_pred'] = rf.predict(df)
    df[elements + '_res'] = df[elements + '_pred'] - df[elements]
    
    
#     fig = px.scatter(
#     df_ref, x=elements, y=elements + '_pred',
#     marginal_x='histogram', marginal_y='histogram',
#     color='split', trendline='ols'
#     )
#     fig.update_traces(histnorm='probability', selector={'type':'histogram'})
#     fig.add_shape(
#         type="line", line=dict(dash='dash'),
#         x0=y.min(), y0=y.min(),
#         x1=y.max(), y1=y.max()
#     )

#     fig.show()

#     fig = px.scatter(
#     df_ref, x=elements + '_pred', y=elements + '_res',
#     marginal_y='violin',
#     color='split', trendline='ols')
#     fig.show()
    
    return rf

def create_Model_Regression(labels):

    df['labels'] = labels
    df_all = df
    train_idx, test_idx = train_test_split(df.index, test_size=.2)
    df['split'] = 'train'
    df.loc[test_idx, 'split'] = 'test'
    print(df.columns)
    y = df_all['labels']
    X = df[['SIO2', 'TIO2', 'AL2O3', 'CR2O3', 'MNO', 'MGO', 'CAO', 'NA2O', 'K2O',
             'F', 'CL', 'NIO', 'ZNO', 'FEOT']]
    
    
    train_features = df.loc[train_idx, ['SIO2', 'TIO2', 'AL2O3', 'CR2O3', 'MNO', 'MGO',
                                        'CAO', 'NA2O', 'K2O','F', 'CL', 'NIO', 'ZNO', 'FEOT']]
    df_all['labels'] = y
    train_labels = df_all.loc[train_idx, 'labels']
    
    rf = RandomForestRegressor(n_estimators = 100)
    # Train the model on training data
    rf.fit(train_features, train_labels);
    # Use the forest's predict method on the test data
    predictions = rf.predict(X)
    # Calculate the absolute errors
    errors = abs(predictions - y)
    # Print out the mean absolute error (mae)
    print('Mean Absolute Error:', round(np.mean(errors), 2), 'Mols.')
    
    df[elements] = df_ref[elements]
    df[elements + '_pred'] = rf.predict(df[['SIO2', 'TIO2', 'AL2O3', 'CR2O3', 'MNO', 'MGO', 'CAO', 'NA2O', 'K2O',
             'F', 'CL', 'NIO', 'ZNO', 'FEOT']])
#     df[elements + '_res'] = df[elements + '_pred'] - df[elements]
    fig = px.scatter(
    df, x=elements, y=elements + '_pred',
    marginal_x='histogram', marginal_y='histogram',
    color='split', trendline='ols'
    )
    fig.update_traces(histnorm='probability', selector={'type':'histogram'})
    fig.add_shape(
        type="line", line=dict(dash='dash'),
        x0=y.min(), y0=y.min(),
        x1=y.max(), y1=y.max()
    )

    fig.write_image('./Anf_regressions/'+elements+'.jpeg')
    return rf
    
    

In [244]:
df_ref = df_anf.filter(regex='.REF$', axis=1)
# print(df_ref.columns)
remove = ['T.TI.REF', 'C.NI.REF', 'B.FE2.REF', 'A.LI.REF', 'A.CA.REF', 
          'W.CL.REF','W.I.REF', 'W.SUM.REF', 'A.SUM.REF', 'B.SUM.REF',
          'SUM C.REF', 'T.SUM.REF']
for col in remove:
    df_ref.pop(col)
df_ref= df_ref.fillna(0)
models_regression = {}
df_ref

,T.SI.REF,T.AL.REF,C.AL.REF,C.TI.REF,C.FE3.REF,C.CR.REF,C.ZN.REF,C.MN2.REF,C.MG.REF,C.FE2.REF,...,B.MN2.REF,B.MG.REF,B.LI.REF,B.CA.REF,B.NA .REF,A.NA.REF,A.K.REF,W.OH.REF,W.F.REF,W.O.REF
0,8.000,0.000,1.760,0.000,0.200,0.000,0.010,0.040,2.010,1.000,...,0.00,0.02,1.97,0.000,0.010,0.020,0.000,1.930,0.070,0.0
1,8.000,0.000,1.340,0.000,0.600,0.000,0.010,0.040,2.350,0.660,...,0.00,0.00,1.98,0.000,0.020,0.030,0.000,1.910,0.090,0.0
2,8.000,0.000,1.300,0.000,0.680,0.000,0.010,0.060,2.320,0.620,...,0.00,0.00,1.92,0.000,0.080,0.020,0.000,1.870,0.130,0.0
3,8.000,0.000,1.330,0.000,0.540,0.000,0.010,0.040,2.420,0.720,...,0.00,0.06,1.93,0.000,0.010,0.040,0.000,1.900,0.100,0.0
4,7.883,0.116,0.000,0.067,1.415,0.000,0.057,0.102,0.000,2.901,...,0.00,0.00,0.00,0.000,2.000,0.680,0.316,0.855,1.145,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,7.980,0.020,1.970,0.010,0.000,0.000,0.020,0.080,1.100,1.160,...,0.00,0.00,1.92,0.030,0.050,0.680,0.000,0.970,1.030,0.0
62,8.000,0.000,1.160,0.030,0.630,0.000,0.040,0.090,1.770,0.380,...,0.00,0.00,0.00,0.010,1.990,0.650,0.430,0.530,1.470,0.0
63,7.214,0.786,0.427,0.015,0.296,0.273,0.000,0.000,3.757,0.181,...,0.00,0.00,0.00,1.221,0.779,0.848,0.043,1.674,0.026,0.3
64,7.390,0.610,0.090,0.080,0.310,0.000,0.000,0.020,3.290,1.190,...,0.03,0.00,0.00,1.180,0.790,0.550,0.320,0.770,1.230,0.0


In [245]:
for elements in df_ref.columns:
    labels = df_ref[elements]
    print(elements)
    models_regression[elements] = create_Model_Regression(labels)
    

T.SI.REF
Index(['SIO2', 'TIO2', 'AL2O3', 'CR2O3', 'MNO', 'MGO', 'CAO', 'NA2O', 'K2O',
       'F', 'CL', 'NIO', 'ZNO', 'FEOT', 'labels', 'split'],
      dtype='object')
Mean Absolute Error: 0.02 Mols.
T.AL.REF
Index(['SIO2', 'TIO2', 'AL2O3', 'CR2O3', 'MNO', 'MGO', 'CAO', 'NA2O', 'K2O',
       'F', 'CL', 'NIO', 'ZNO', 'FEOT', 'labels', 'split', 'T.SI.REF',
       'T.SI.REF_pred'],
      dtype='object')
Mean Absolute Error: 0.02 Mols.
C.AL.REF
Index(['SIO2', 'TIO2', 'AL2O3', 'CR2O3', 'MNO', 'MGO', 'CAO', 'NA2O', 'K2O',
       'F', 'CL', 'NIO', 'ZNO', 'FEOT', 'labels', 'split', 'T.SI.REF',
       'T.SI.REF_pred', 'T.AL.REF', 'T.AL.REF_pred'],
      dtype='object')
Mean Absolute Error: 0.04 Mols.
C.TI.REF
Index(['SIO2', 'TIO2', 'AL2O3', 'CR2O3', 'MNO', 'MGO', 'CAO', 'NA2O', 'K2O',
       'F', 'CL', 'NIO', 'ZNO', 'FEOT', 'labels', 'split', 'T.SI.REF',
       'T.SI.REF_pred', 'T.AL.REF', 'T.AL.REF_pred', 'C.AL.REF',
       'C.AL.REF_pred'],
      dtype='object')
Mean Absolute Error: 0.01 Mols